In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [2]:
day = 150926
data_type = 'ctime'
detector_name = 'n4'
echan = 2
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
year = int('20' + str(day)[0:2])
date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))


data = calc.curve_fit_plots(day, detector_name, echan)
residuals = data[0]
counts = data[1]
fit_curve = data[2]
cgb = data[3]
magnetic = data[4]
earth_ang_bin = data[5]
sun_ang_bin = data[6]
crab_ang_bin = data[7]
scox_ang_bin = data[8]
cygx_ang_bin = data[9]
plot_time_bin = data[10]
plot_time_sat = data[11]

In [15]:
rms = np.sqrt(np.mean(np.square(np.fabs(residuals))))

In [16]:
print rms

32.0591908749


In [ ]:
saa_exits = [0]
for i in range(1, len(counts)):
    if np.logical_and(counts[i-1] == 0, counts[i] != 0):
        print i
        saa_exits.append(i)
saa_exits = np.array(saa_exits)

if saa_exits[1] - saa_exits[0] < 10:
    saa_exits = np.delete(saa_exits, 0)

In [ ]:
saa_entries = [0]
for i in range(1, len(counts)-1):
    if np.logical_and(counts[i+1] == 0, counts[i] != 0):
        print i
        saa_entries.append(i)
saa_entries = np.array(saa_entries)

if saa_entries[1] - saa_entries[0] < 10:
    saa_entries = np.delete(saa_entries, 0)

In [ ]:
#residuals[np.fabs(residuals) < 4*rms] = 0

In [ ]:
print residuals

In [ ]:
fourier = np.fft.fft(residuals[22312:167292])
freq = np.fft.fftfreq(len(fourier), plot_time_bin[22314] - plot_time_bin[22313])

In [ ]:
print fourier

In [ ]:
#plt.plot(plot_time_bin[22312:167292], residuals[22312:167292], 'b-')
plt.plot(freq, np.abs(fourier), 'b-')
#plt.plot(freq, np.angle(fourier), 'ro')
            
plt.xlabel('Time of day in 24h', fontsize=20)
plt.ylabel('Residual noise', fontsize=20)
            
plt.grid()
            
plt.title(data_type + '-countrate-fit residuals of the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)
            
#plt.xlim([-0.5, 24.5])
#if counts.all == total_rate.all:
#    plt.ylim([-600, 600])
#else:
#plt.ylim([-300, 300])
            

            

plt.show()

In [ ]:
import scipy.fftpack
N = 167292-22312
T = plot_time_bin[22314] - plot_time_bin[22313]

yf = scipy.fftpack.fft(residuals[22312:167292])
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

fig, ax = plt.subplots()
ax.plot(xf, 2.0/N * np.abs(yf[:N/2]))
plt.show()

In [27]:
resids = residuals[22312:167292]
times = plot_time_bin[22312:167292]

binning = 101
#if binning == 5:
#    binned_res = resids[0::5] + resids[1::5] + resids[2::5] + resids[3::5] + resids[4::5]
#    binned_times = times[2::5]
#elif binning == 11:
#    binned_res = resids[0::11] + resids[1::11] + resids[2::11] + resids[3::11] + resids[4::11] + resids[5::11] + resids[6::11] + resids[7::11] + resids[8::11] + resids[9::11] + resids[10::11]
#    binned_times = times[5::11]
    
binned_res = resids[0::binning]
binned_times = times[int(binning/2)::binning]
for i in range(1, binning):
    if len(binned_res) != len(resids[i::binning]):
        binned_res = np.delete(binned_res, -1)
    binned_res = np.add(binned_res, resids[i::binning])

if len(binned_res) < len(binned_times):
    binned_times = np.delete(binned_times, -1)

print len(binned_res)
print len(binned_times)

1435
1435


In [28]:
import scipy.fftpack
N = len(binned_res)
T = binned_times[1] - binned_times[0]

yf = scipy.fftpack.fft(binned_res)
xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

fig, ax = plt.subplots()
ax.plot(xf, 2.0/N * np.abs(yf[:N/2]))
plt.show()

In [29]:
plt.plot(binned_times, binned_res, 'b-')
#plt.plot(freq, np.abs(fourier), 'b-')
#plt.plot(freq, np.angle(fourier), 'ro')
            
plt.xlabel('Time of day in 24h', fontsize=20)
plt.ylabel('Residual noise', fontsize=20)
            
plt.grid()
            
plt.title(data_type + '-countrate-fit residuals of the ' + detector_name + '-detector on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=30)
            

plt.show()

In [24]:
rms = np.sqrt(np.mean(np.square(np.fabs(binned_res))))

print rms/binning

32.0591908749
